In [1]:
import math
import random
import time

In [2]:
class Problem:
    """Esta es una clase abstracta para un problema.
    No se supone que debamos crear objetos de esta
    clase directamente. La idea es que otras clases
    hereden de esta para crear clases de problemas
    más especeificos."""
    
    def random_solution(self):
        """Este metodo sirve para dar una solución random
        de nuestro problema."""
        raise NotImplementedError
    
    def neighbors_of(self,state):
        """Calcula los vecinos de un estado."""
        raise NotImplementedError
    
    def random_neighbor(self,state):
        """De los vecinos de un estado escoge a un 
        vecino al azar."""
        return self.random_choice(state)
    
    def crossover(self,state):
        """Combina a dos estados dados en uno solos
        Eejemplo: s1 = [1,2,3,4], s2=[5,6,7,8], los 
        resultados posibles son [1,6,7,8],[1,2,7,8]
        y [1,2,3,8]"""
        raise NotImplementedError
    
    def domain_size(self):
        """Calcula el tamaño del dominio para asegurar
        que prodremos calcular ciertos priceso en un tiempo
        razonable"""
        raise NotImplementedError
    
    def all_domain(self):
        raise NotImplementedError
    
    def value(self, state):
        """En distintos problemas, cada estado tiene un valor asociado.
        Este método sirve para obtener este valor."""
        raise NotImplementedError
        

In [3]:
# Funcion que me ayuda a calculat todos los elementos del dominio
# Lo que hace esta funcion toma una lista, que va a contener todos los vuelos posibles
# de n personas. x1 y x2, es el numero de vuelos disponibles para la persona n+1 de ida y vuelta
# Entonces por cada elemento en la lista va a agregar x1*x2 elementos nuevos que son las posibles 
# nuevas coluciones para el caso n+1.
def todo(lista,x1,x2):
    i = 0
    j = 0
    lista_2 = []
    if len(lista) != 0:
        for element in lista:
            for i in range(x1):
                for j in range(x2):
                    lista_2.append(element + [i,j])
                    j+=1
                i+=1
    
    else:
        for i in range(x1):
            for j in range(x2):
                lista_2.append([i,j])
                j+=1
            i+=1
    return lista_2


In [4]:
class Viaje(Problem):
    # Definimos las propiedades  de nuestra clase
    def __init__(self,people, schedule, airports):
        self.schedule = schedule
        self.airports = airports
        self.people = people
        self.destination = 'LGA'
        domain = []
        family = len(people)
        for i in range(family):
            origin = self.people[i][1]
            num_arr_flights = len(self.schedule[(origin,self.destination)])-1
            num_dep_flights = len(self.schedule[(self.destination,origin)])-1
            domain = domain + [(0,num_arr_flights)] + \
                              [(0,num_dep_flights)]
        self.domain  = domain
        

    # Obtenemos el tiempo en minutos entre 2 horas dadas
    def get_minutes(self, t):
        x = time.strptime(t, '%H:%M')
        h = x.tm_hour
        m = x.tm_min
        return 60 * h + m
    
    def value(self,s):
        # contamos el precio total de cada vuelo (ida y regreso)
        total_price = 0
        
        # nos interesa conocer el tiempo de llegada a NY mas tarde
        # y el tiempo de salida de NY mas temprano.
        latest_arrival = 0
        earliest_departure = 24 * 60
        earliest_arrival = 24 * 60
        latest_departure = 0
        
        for i in range(len(s) // 2):
            origin = self.people[i][1]
            out_flight = self.schedule[(origin, self.destination)][s[2*i]]
            ret_flight = self.schedule[(self.destination, origin)][s[2*i+1]]
            
            total_price += out_flight[2] # vuelo de ida
            total_price += ret_flight[2] # vuelo de regreso

            out_arrival = self.get_minutes(out_flight[1])
            ret_departure = self.get_minutes(ret_flight[0])
            
            earliest_arrival = min(earliest_arrival, out_arrival)
            latest_arrival = max(latest_arrival, out_arrival)
            earliest_departure = min(earliest_departure, ret_departure)
            latest_departure = max(latest_departure, ret_departure)
        
        if latest_arrival < earliest_departure + 60:
            family_time = (math.e ** (earliest_departure - latest_arrival - 60))+ 1000
        else:
            family_time = latest_arrival - earliest_departure

        out_waiting_time = latest_arrival - earliest_arrival
        ret_waiting_time = latest_departure - earliest_departure
        family_time = earliest_departure - latest_arrival
        
        return (
            total_price + 
            out_waiting_time + 
            ret_waiting_time -
            family_time
        )
    
    # Genera un saloción random
    def random_solution(self):
        return [random.randint(r[0], r[1]) for r in self.domain]

    
    # pone en una lista a todos los vecinos de un estado
    def neighbors_of(self, state):
        neighbors = []
        for i in range(len(self.domain)):
            if state[i] > self.domain[i][0]:
                neighbors.append(state[0:i] + [state[i] - 1] + state[i+1:])
            if state[i] < self.domain[i][1]:
                neighbors.append(state[0:i] + [state[i] + 1] + state[i+1:])
        return neighbors
    
    # Combinamos dos estados 
    def crossover(self, s1, s2):
        i = random.randint(1, len(s1)-2)
        return s1[:i] + s2[i:]
    
    # Escoge a un vecino de un estado al azar.
    def random_neighbor(self,s):

        #Hacemos una lista de indices de todos los vuelos que se van a realizar.

        all_flights = [i for i in range(2*len(self.people))]

        # Hacemos una sublista con los indices que el estado en ese indice sea 0 o sea el mayor vuelo posible.

        one_neighbor = [i for i in all_flights if s[i] == 0 or  (i % 2 == 0 and s[i] == len(self.schedule \
                            [(self.people[ i//2][1],self.destination)])-1) or (i % 2 == 1 and s[i] == len(self.schedule \
                            [(self.destination,self.people[ i//2][1])])-1)]
        
        # Tomo el complemento de la sdublista anterior con respecto a la primera.

        two_neighbors = [item for item in all_flights if item not in one_neighbor]
        
        # Vamos a tener que la cantidad de vecinos que tenga el estado s es el siguiente.

        num_neighbors = 2*len(two_neighbors) + len(one_neighbor)

        # Tomamos un numero al azar enter 0 y 1

        x = random.random()

        # Como queremos hacer una random uniforme tenemos que separar el intervalo en 2 pedazos, que uno represente cuando
        # el indice tiene 2 vecinos o uno, en este caso tomamos el caso que tenga 2 vecinos.

        if x < 2*len(two_neighbors) / num_neighbors:
            y= random.choice(two_neighbors)
            s[y] = random.choice([s[y]+1,s[y]-1])

        # Si no tiene 2 vecinos, entonce tiene un vecino.

        else:
            y = random.choice(one_neighbor)
            if s[y] == 0:
                s[y] = 1
            elif y % 2 == 0:
                s[y] = len(self.schedule[(self.people[ y//2][1],self.destination)])-2
            else:
                s[y] = len(self.schedule[(self.destination,self.people[ y//2][1])])-2
        return s

    # Esto es solamente para saber calcular la cardinalidad del dominio, y verificar si podemos calcular 
    # ciertas cosas con mi computadora
    
    def domain_size(self):
        size = 1
        for i in range(len(self.people)):
            size = len(self.schedule[(self.people[ i//2][1],self.destination)]) * \
                   len(self.schedule[(self.destination,self.people[i//2][1])]) * size
        if size > 1000000:
            return True
        else:
            return False
    
    def all_domain(self):
        while not self.domain_size:
            all = []
            for i in range(len(self.people)):
                flight_person_ret = len(self.schedule[(self.people[i//2][1],self.destination)])
                flight_person_out = len(self.schedule[(self.destination,self.people[i//2][1])])
                all = todo(all,flight_person_ret,flight_person_out)
            return all
        return "El dominio es demasiado grande"

In [5]:
class OptimazationMethod:
    def solve(slef, Problem):
        raise NotImplementedError
    


In [6]:
class ExhaustiveSearch(OptimazationMethod):
    def solve(self,Problem):
        if Problem.domain_size():
            return "Impossible to calculate with this optimization method in this computer."
        else:
            solution = None
            min_cost = float('inf')
            for element in Problem.all_domain():
                if Problem.value(element) < min_cost:
                    solution = element
                    min_cost = Problem.value(solution)
            return solution


In [7]:
class SolveRandomly(OptimazationMethod):
    def __init__(self, repeats: int = 1000):
        self.repeats = repeats
        
    def solve(self,Problem,):
        best_cost = float('inf')
        best_sol = None
    
        for _ in range(self.repeats):
            s = Problem.random_solution()
            c = Problem.value(s)
            if c <= best_cost:
                best_cost = c
                best_sol = s
    
        return best_sol

In [8]:
class HillClimbing(OptimazationMethod):
    def solve(self, Problem):
        solution = Problem.random_solution()
        
        while True:
            neighbors = Problem.neighbors_of(solution)
            if len(neighbors) == 0:
                return solution
            cost = Problem.value(solution)
            best_neighbor = min(neighbors, key=Problem.value)
            neighbor_cost = Problem.value(best_neighbor)
            
            if cost < neighbor_cost:  
                return solution
            
            solution = best_neighbor

In [9]:
class Annealing(OptimazationMethod):
    def __init__(self,Ti=10000,Tf=.1,alpha=.95):
        self.Ti = Ti
        self.Tf = Tf
        self.alpha = alpha

    def solve(self,Problem):
        solution = Problem.random_solution()
        cost = Problem.value(solution)
        T=self.Ti
        while T > self.Tf:
            if Problem.neighbors_of(solution) == 0:
                return solution
            neighbor = Problem.random_neighbor(solution)
            neighbor_cost = Problem.value(neighbor)
            diff = cost - neighbor_cost
            if diff > 0 or random.random() < math.exp(diff / T):
                solution = neighbor
                cost = neighbor_cost
            T = self.alpha*T
        return solution
        

In [10]:
class Evoling(OptimazationMethod):
    def __init__(self, pop_size=50, mut_prob=0.2, elite=0.2, epochs=100):
        self.pop_size = pop_size
        self.mut_prob = mut_prob
        self.elite = elite
        self.epochs = epochs

    def solve(self, Problem):
        pop = [Problem.random_solution() for _ in range(self.pop_size)]
        top_elite = int(self.elite * self.pop_size)
        
        for epoch in range(self.epochs):
            pop.sort(key=Problem.value)
            best = pop[:top_elite]
            while len(best) < self.pop_size:
                if random.random() < self.mut_prob:
                    best.append(Problem.random_neighbor(
                        best[random.randint(0, top_elite-1)]
                    ))
                else:
                    best.append(Problem.crossover(
                        best[random.randint(0, top_elite-1)],
                        best[random.randint(0, top_elite-1)],
                    ))
            pop = best
        pop.sort(key=Problem.value)
        return pop[0]

In [11]:
people = [('Seymour', 'BOS'),
          ('Franny' , 'DAL'),
          ('Zooey'  , 'CAK'),
          ('Walt'   , 'MIA'),
          ('Buddy'  , 'ORD'),
          ('Les'    , 'OMA')]


In [12]:
AIRPORT_PATH = "./data/airport-codes.txt"
SCHEDULE_PATH = "./data/schedule.txt"

In [13]:
def load_airports(airports):
        airport = {}
        with open(airports) as f:
            f.readline()
            for line in f:
                cols = line.strip().split(',')
                iata = cols[9]
                name = cols[2]
                region = cols[6]
                municipality = cols[7]
                airport[iata] = (
                    name,
                    region,
                    municipality
                )
        return airport

In [14]:
def load_flights(path):
    flights = {}
    with open(path) as f:
        for line in f:
            origin, dest, t_depart, t_arrive, price = line.strip().split(',')
            flights.setdefault((origin, dest), [])
            flights[(origin, dest)].append((t_depart, t_arrive, int(price)))
    return flights

In [15]:
NewYork = Viaje(people, load_flights(SCHEDULE_PATH),load_airports(AIRPORT_PATH))

In [16]:
print(SolveRandomly().solve(NewYork))

[3, 4, 4, 4, 1, 8, 3, 4, 1, 8, 3, 6]


In [17]:
print(HillClimbing().solve(NewYork))

[3, 3, 2, 2, 1, 3, 2, 1, 4, 6, 2, 6]


In [18]:
print(ExhaustiveSearch().solve(NewYork))

Impossible to calculate with this optimization method in this computer.


In [19]:
print(Annealing().solve(NewYork))

[1, 4, 4, 6, 1, 3, 8, 4, 7, 7, 0, 7]


In [20]:
print(Evoling().solve(NewYork))

[1, 6, 2, 3, 1, 5, 2, 6, 1, 3, 2, 6]


In [21]:
NewYork.random_neighbor([0,2,3,4,5,0,7,8,5,0,1,2])

[0, 2, 3, 4, 5, 0, 7, 8, 6, 0, 1, 2]